## Study time dependence of a transient source

Look at month16 ROi 1541:
<pre>
name               dist        ra       dec        TS eflux(eV)    index   energy   beta/b
TSh16-1238          1.4    15.247   -68.302        13       6.5*    2.34*     697     0
PSR J0101-6422      2.5    15.287   -64.368        19       5.6*    1.13     1826     1
TSh16-1223          2.9    16.521   -69.772        18       9.1*    1.64*     950     0
</pre>

I want to examine the time structure of each of these three sources.

First CD into the folder for the model.

In [ ]:
%matplotlib inline
month_number=16
month_name='month%02d' % month_number
roi_number=1541
os.chdir(os.path.expandvars('$FERMI/skymodels/P301_monthly/%s' % month_name))

Then get the FT1 file

In [ ]:
import pyfits
ft1_file='/nfs/farm/g/glast/g/catalog/P8_P301/Source/P301_Source_%03d_zmax100.fits'% month_number
ft1 = pyfits.open(ft1_file)
print ft1.info()
events = ft1[1].data

In [ ]:
ra = events.RA
dec = events.DEC
time = events.TIME
energy = events.ENERGY

In [ ]:
import pickle
sourcedf = pickle.load(open('sources.pickle'))

In [ ]:
def lightcurve(source_name):
    adir = sourcedf.ix[source_name].skydir
    cut = (np.abs(ra-adir.ra())<10) & (np.abs(dec-adir.dec())<10)& (energy>400) 
    sdir = map(SkyDir, np.array(ra[cut],float), np.array(dec[cut],float))
    diffs = np.asarray(np.degrees( [ adir.difference(b) for b in sdir]))        
    day=(time[cut]-time.min())/(24*3600.)
    
    fig, axx = plt.subplots(1,2, figsize=(12,4))

    ax = axx[0]
    ax.hist(diffs**2, np.linspace(0,4,41), histtype='step');
    ax.grid(True, alpha=0.5)
    plt.setp(ax, xlabel='distance**2 [deg**2]')

    ax=axx[1]
    ax.hist(day[diffs<4], np.linspace(0, 30,31), histtype='step', log=True);
    ax.hist(day[diffs<0.5], np.linspace(0, 30, 31), histtype='step', log=True);
    ax.grid(True, alpha=0.5)
    plt.setp(ax, ylim=(0.8,None),xlabel='day of month' )
    fig.suptitle('%s for month %d' % (source_name, month_number) )

In [ ]:
s=sourcedf
s[(s.psr) & (s.ts>1000)]['ts ra dec glat '.split()]

In [ ]:
ts_sources = np.asarray([s.startswith('TS') for s in sourcedf.index], bool)
cands = sourcedf.ix[ts_sources & (sourcedf.ts>10) & (np.abs(sourcedf.glat)>10)\
                    & (sourcedf.locqual<8)]['ts ra dec a locqual'.split()]
cands.sort_index()

